# Chatbot System Design

We demonstrate chatbot design from intent and self-service definition to multi-step input preprocessing and response generation.

### Features
- Regex-based pattern matching for intent recognition
- Text preprocessing with tokenization, punctuation removal, and stemming
- Response variation through random selection
- Template substitution with customer name and order details
- Context management for multi-turn dialogs

### Application Domain: Pizza Ordering Self-Service Bot

The bot handles the following intents:

- **Greeting/Welcome**: Friendly introductions and welcomes
- **Menu Inquiry**: Display available pizzas, sizes, sides, and drinks with prices
- **Place Order**: Take pizza orders with item extraction, size selection, and name capture
- **Order Status Check**: Check on the status of placed orders
- **Exit/Goodbye**: Graceful conversation termination (type `exit` or `quit`)



## 1. Dependencies Setup

We use the following libraries:
- **re**, **json**, **random**: Standard libraries for regex, JSON handling, and randomization
- **nltk**: Natural Language Toolkit for text preprocessing (stemming, tokenization)

In [1]:
import subprocess
import sys

try:
    import nltk
except ImportError:
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'nltk'])
    import nltk

nltk.download('punkt', quiet=True)
nltk.download('punkt_tab', quiet=True)
nltk.download('wordnet', quiet=True)
nltk.download('stopwords', quiet=True)

print("Dependencies installed and ready!")

Dependencies installed and ready!


## 2. Import Libraries

In [2]:
import json
import re
import random
import string
from typing import Dict, List, Optional, Tuple, Any

from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

print("Libraries imported successfully!")

Libraries imported successfully!


## 3. Load JSON Intents

### JSON Schema Structure

The `intents.json` file contains only intent definitions - all configuration is handled in the notebook code.

```json
{
  "intents": [
    {
      "tag": "intent_name",           // Unique identifier for the intent
      "patterns": [                    // List of regex patterns to match user input
        "^(hi|hello)\\b",             // Regex with case-insensitive matching
        "\\bshow.*menu\\b"
      ],
      "responses": [                   // List of response templates (randomly selected)
        "Hello, {name}!",             // Supports {placeholder} substitution
        "Hi there!"
      ],
      "responses_no_order": [...],    // Alternative responses when no order exists (order_status)
      "responses_no_name": [...],     // Alternative responses when name unknown (goodbye)
      "extract": {                     // Entity extraction rules (order intent only)
        "item": ["pepperoni", ...],
        "size": ["small", "medium", "large"]
      },
      "name_prompt": [...],           // Prompts when asking for customer name
      "confirm_item_prompt": [...],   // Prompts when asking for pizza type
      "prices": {                      // Base prices for items (order intent only)
        "pepperoni": 12,
        "margherita": 10
      },
      "size_modifiers": {              // Price adjustments by size (order intent only)
        "small": 0,
        "medium": 2,
        "large": 4
      }
    }
  ]
}
```

### Intent Types

| Intent | Purpose | Special Fields |
|--------|---------|----------------|
| `greeting` | Welcome users | - |
| `menu` | Show available items | - |
| `order_status` | Check order progress | `responses_no_order` |
| `order` | Place pizza orders | `extract`, `name_prompt`, `confirm_item_prompt`, `prices`, `size_modifiers` |
| `goodbye` | End conversation | `responses_no_name` |
| `fallback` | Handle unknown input | No patterns (catches everything else) |

In [3]:
# Configuration (defined in notebook, not JSON)
CONFIG = {
    # Order defaults
    'default_size': 'Medium',

    # Pattern matching
    'case_insensitive': True,

    # Name extraction
    'max_name_words': 2,

    # Preprocessing
    'preprocessing': {
        'expand_contractions': True,
        'lowercase': True,
        'remove_punctuation': True,
        'stem': True,
        'remove_stopwords': False,
    },

    # Messages
    'empty_input_message': "Please say something! I'm here to help.",
    'error_message': "Oops! Something went wrong. Let's try again.",
    'name_only_response': "Nice to meet you, {name}! How can I help you today? Would you like to see our menu or place an order?",
    'order_error_message': "Sorry, I'm having trouble processing orders right now.",
    'fallback_goodbye': "Thank you for visiting Pizza Palace! Goodbye!",
    'interrupt_goodbye': "Goodbye! Thanks for visiting Pizza Palace!",
    'no_match_response': "I'm sorry, I didn't understand that. Could you try rephrasing?",
}

def load_intents(file_path: str = "intents.json") -> List[Dict]:
    """Load intents from a JSON file."""
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        print(f"Loaded {len(data['intents'])} intents from {file_path}")
        return data['intents']
    except FileNotFoundError:
        print(f"Error: Could not find {file_path}")
        raise
    except json.JSONDecodeError as e:
        print(f"Error: Invalid JSON in {file_path}: {e}")
        raise

intents = load_intents()

print("\nConfiguration:")
for key, value in CONFIG.items():
    if isinstance(value, dict):
        print(f"  {key}:")
        for k, v in value.items():
            print(f"    {k}: {v}")
    else:
        print(f"  {key}: {value}")

print("\nLoaded intents:")
for intent in intents:
    print(f"  - {intent['tag']}: {len(intent['patterns'])} patterns, {len(intent['responses'])} responses")

Loaded 6 intents from intents.json

Configuration:
  default_size: Medium
  case_insensitive: True
  max_name_words: 2
  preprocessing:
    expand_contractions: True
    lowercase: True
    remove_punctuation: True
    stem: True
    remove_stopwords: False
  empty_input_message: Please say something! I'm here to help.
  error_message: Oops! Something went wrong. Let's try again.
  name_only_response: Nice to meet you, {name}! How can I help you today? Would you like to see our menu or place an order?
  order_error_message: Sorry, I'm having trouble processing orders right now.
  fallback_goodbye: Thank you for visiting Pizza Palace! Goodbye!
  interrupt_goodbye: Goodbye! Thanks for visiting Pizza Palace!
  no_match_response: I'm sorry, I didn't understand that. Could you try rephrasing?

Loaded intents:
  - greeting: 5 patterns, 5 responses
  - menu: 5 patterns, 2 responses
  - order_status: 5 patterns, 4 responses
  - order: 5 patterns, 4 responses
  - goodbye: 1 patterns, 4 response

## 4. Text Preprocessing

Preprocessing improves pattern matching accuracy by normalizing user input. The following steps are applied:

1. **Contraction Expansion**: Expands contractions to full forms (e.g., "I'd" -> "I would", "What's" -> "What is")
2. **Lowercasing**: Converts all text to lowercase for case-insensitive matching
3. **Punctuation Removal**: Strips punctuation marks that might interfere with matching
4. **Tokenization**: Splits text into individual words/tokens
5. **Stemming**: Reduces words to their root form using Porter Stemmer (e.g., "ordering" -> "order", "pizzas" -> "pizza")
6. **Stop-word Removal** (optional): Removes common words like "the", "a", "is"

**Note**: Stemming produces morphological roots (e.g., "please" -> "pleas"), which may look unusual but helps match word variations consistently. The original input is preserved for display; preprocessing is only used internally for pattern matching.

In [4]:
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

CONTRACTIONS = {
    "i'd": "i would", "i'll": "i will", "i'm": "i am", "i've": "i have",
    "you'd": "you would", "you'll": "you will", "you're": "you are", "you've": "you have",
    "he'd": "he would", "he'll": "he will", "he's": "he is",
    "she'd": "she would", "she'll": "she will", "she's": "she is",
    "it's": "it is", "it'll": "it will",
    "we'd": "we would", "we'll": "we will", "we're": "we are", "we've": "we have",
    "they'd": "they would", "they'll": "they will", "they're": "they are", "they've": "they have",
    "that's": "that is", "what's": "what is", "where's": "where is",
    "who's": "who is", "how's": "how is",
    "can't": "cannot", "couldn't": "could not", "didn't": "did not",
    "doesn't": "does not", "don't": "do not", "hadn't": "had not",
    "hasn't": "has not", "haven't": "have not", "isn't": "is not",
    "wasn't": "was not", "weren't": "were not", "won't": "will not",
    "wouldn't": "would not", "let's": "let us",
}

def expand_contractions(text: str) -> str:
    """Expand contractions (e.g., \"I'd\" -> \"I would\")."""
    text_lower = text.lower()
    for contraction, expansion in CONTRACTIONS.items():
        text_lower = re.sub(r'\b' + re.escape(contraction) + r'\b', expansion, text_lower)
    return text_lower

def preprocess_text(text: str) -> str:
    """
    Preprocess text for pattern matching using CONFIG settings.

    Steps: expand contractions -> lowercase -> remove punctuation -> tokenize -> stem
    """
    prep = CONFIG['preprocessing']

    if prep['expand_contractions']:
        text = expand_contractions(text)
    if prep['lowercase']:
        text = text.lower()
    if prep['remove_punctuation']:
        text = re.sub(r'[^\w\s]', ' ', text)

    tokens = text.split()

    if prep['remove_stopwords']:
        important_words = {'i', 'my', 'me', 'want', 'like', 'order', 'have', 'get', 'can', 'could', 'would'}
        tokens = [t for t in tokens if t not in stop_words or t in important_words]
    if prep['stem']:
        tokens = [stemmer.stem(token) for token in tokens]

    return ' '.join(tokens)

def tokenize_input(text: str) -> List[str]:
    """Simple tokenization for word extraction."""
    text = text.lower()
    text = re.sub(r'[^\w\s]', ' ', text)
    return text.split()

test_inputs = [
    "I'd like to ORDER a PEPPERONI pizza, please!",
    "What's on the menu?",
    "Hello there!",
    "Can't you show me what you've got?"
]

print("Preprocessing examples:")
print(f"Settings: {CONFIG['preprocessing']}")
print("-" * 70)
for test in test_inputs:
    processed = preprocess_text(test)
    print(f"Input:  '{test}'")
    print(f"Output: '{processed}'")
    print()

Preprocessing examples:
Settings: {'expand_contractions': True, 'lowercase': True, 'remove_punctuation': True, 'stem': True, 'remove_stopwords': False}
----------------------------------------------------------------------
Input:  'I'd like to ORDER a PEPPERONI pizza, please!'
Output: 'i would like to order a pepperoni pizza pleas'

Input:  'What's on the menu?'
Output: 'what is on the menu'

Input:  'Hello there!'
Output: 'hello there'

Input:  'Can't you show me what you've got?'
Output: 'cannot you show me what you have got'



## 5. Pattern Matching

The pattern matching system uses regular expressions to identify user intents:

- **Case-insensitive matching** (`re.IGNORECASE`)
- **Word boundaries** (`\b`) to avoid partial matches
- **Optional elements** (`?`) for flexible phrasing
- **Character classes** (`\s`, `\w`) for robust matching
- **Quantifiers** (`*`, `+`) for variable-length patterns

In [5]:
def match_intent(user_input: str, intents_list: List[Dict]) -> Tuple[Optional[Dict], Optional[re.Match]]:
    """Match user input against intent patterns. Returns (intent, match) or (fallback, None)."""
    clean_input = user_input.strip().lower()
    regex_flags = re.IGNORECASE if CONFIG['case_insensitive'] else 0

    for intent in intents_list:
        if intent['tag'] == 'fallback':
            continue
        for pattern in intent['patterns']:
            try:
                match = re.search(pattern, clean_input, regex_flags)
                if match:
                    return intent, match
            except re.error as e:
                print(f"Regex error in pattern '{pattern}': {e}")
                continue

    fallback = next((i for i in intents_list if i['tag'] == 'fallback'), None)
    return fallback, None

def extract_order_details(user_input: str, intent: Dict) -> Dict[str, str]:
    """Extract order details (item, size) from user input."""
    extracted = {'item': None, 'size': None}

    if not intent.get('extract'):
        return extracted

    input_lower = user_input.lower()
    tokens = tokenize_input(user_input)

    if 'item' in intent['extract']:
        for item in intent['extract']['item']:
            if item in input_lower:
                item_map = {
                    'veggie': 'Veggie Supreme', 'veggie supreme': 'Veggie Supreme',
                    'meat': 'Meat Lovers', 'meat lovers': 'Meat Lovers',
                    'bbq': 'BBQ Chicken', 'bbq chicken': 'BBQ Chicken',
                    'margherita': 'Margherita', 'pepperoni': 'Pepperoni', 'hawaiian': 'Hawaiian'
                }
                extracted['item'] = item_map.get(item, item.title())
                break

    if 'size' in intent['extract']:
        for size in intent['extract']['size']:
            if size in tokens:
                extracted['size'] = size.capitalize()
                break

    return extracted

print("Pattern matching tests:")
print(f"Case insensitive: {CONFIG['case_insensitive']}")
test_cases = ["Hello!", "Show me the menu", "I'd like a large pepperoni", "Where is my order?", "exit"]
for test in test_cases:
    intent, match = match_intent(test, intents)
    print(f"  '{test}' -> Intent: {intent['tag'] if intent else 'None'}")

Pattern matching tests:
Case insensitive: True
  'Hello!' -> Intent: greeting
  'Show me the menu' -> Intent: menu
  'I'd like a large pepperoni' -> Intent: order
  'Where is my order?' -> Intent: order_status
  'exit' -> Intent: goodbye


## 6. Conversation Context Management

The context manager serves as the **integration point for business logic**. While the chatbot framework handles conversation flow and intent matching, domain-specific operations are injected here:

- **State tracking**: Customer name, order details, conversation state
- **Business logic**: Price calculation, order validation, inventory checks
- **Data persistence**: Order history, session management

This separation allows the same conversational framework to be adapted to different domains by swapping out the business logic layer.

In [6]:
class ConversationContext:
    """
    Manages conversation state and order context.

    This class is the BUSINESS LOGIC INJECTION POINT - domain-specific operations
    like pricing, validation, and order processing are implemented here, separate
    from the conversational framework (intent matching, response generation).
    """

    def __init__(self):
        self.reset()

    def reset(self):
        self.customer_name: Optional[str] = None
        self.current_order: Dict[str, Any] = {
            'item': None,
            'size': CONFIG['default_size'],
            'price': 0
        }
        self.order_history: List[Dict] = []
        self.awaiting: Optional[str] = None
        self.pending_item: Optional[str] = None
        self.pending_size: Optional[str] = None
        self.greeted: bool = False
        self.has_active_order: bool = False

    def set_name(self, name: str):
        self.customer_name = name.strip().title()

    def set_order(self, item: str, size: str = None):
        self.current_order['item'] = item
        if size:
            self.current_order['size'] = size
        self.has_active_order = True

    # =========================================================================
    # BUSINESS LOGIC: Price calculation
    # This is where domain-specific pricing rules are applied.
    # In a production system, this would connect to a pricing service/database.
    # =========================================================================
    def calculate_price(self, intent: Dict) -> float:
        item = self.current_order['item']
        size = self.current_order['size']

        if not item or 'prices' not in intent:
            return 0

        base_price = intent['prices'][item.lower()]
        size_mod = intent['size_modifiers'].get(size.lower(), 0)

        total = base_price + size_mod
        self.current_order['price'] = total
        return total

    def get_context_values(self) -> Dict[str, str]:
        """Prepare values for response template substitution."""
        return {
            'name': self.customer_name or 'valued customer',
            'item': self.current_order['item'] or 'pizza',
            'size': self.current_order['size'] or CONFIG['default_size'],
            'price': f"${self.current_order['price']:.2f}" if self.current_order['price'] else '$0.00'
        }

context = ConversationContext()
print("Conversation context manager initialized.")

Conversation context manager initialized.


## 7. Response Generation

Response generation includes:
- **Random selection** from multiple response templates
- **Template substitution** using `{placeholder}` syntax
- **Context-aware responses** based on conversation state

In [7]:
def generate_response(intent: Dict, context: ConversationContext, user_input: str) -> str:
    """Generate a response based on matched intent and context."""
    tag = intent['tag']

    if context.awaiting == 'name':
        return handle_name_input(user_input, context, intent)
    if context.awaiting == 'item':
        return handle_item_input(user_input, context, intent)
    if context.awaiting == 'name_for_status':
        return handle_name_for_status(user_input, context)

    if tag == 'greeting':
        context.greeted = True
        return random.choice(intent['responses'])
    elif tag == 'menu':
        return random.choice(intent['responses'])
    elif tag == 'order':
        return handle_order_intent(intent, context, user_input)
    elif tag == 'order_status':
        return handle_status_intent(intent, context)
    elif tag == 'goodbye':
        return handle_goodbye_intent(intent, context)
    elif tag == 'fallback':
        return random.choice(intent['responses'])

    return random.choice(intent['responses'])

def handle_name_input(user_input: str, context: ConversationContext, order_intent: Dict) -> str:
    """Handle user providing their name."""
    clean_input = re.sub(r"[^\w\s]", '', user_input).strip()
    words = clean_input.split()

    skip_words = {'my', 'name', 'is', 'i', 'am', 'call', 'me', 'it', "it's", 'the'}
    name_words = [w for w in words if w.lower() not in skip_words]

    max_words = CONFIG['max_name_words']
    name = ' '.join(name_words[:max_words]).title() if name_words else clean_input.title()
    context.set_name(name)
    context.awaiting = None

    if context.pending_item:
        context.set_order(context.pending_item, context.pending_size or CONFIG['default_size'])
        order_intent_obj = next((i for i in intents if i['tag'] == 'order'), None)
        if order_intent_obj:
            context.calculate_price(order_intent_obj)
        context.pending_item = None
        context.pending_size = None
        values = context.get_context_values()
        return random.choice(order_intent_obj['responses']).format(**values)

    return CONFIG['name_only_response'].format(name=name)

def handle_name_for_status(user_input: str, context: ConversationContext) -> str:
    """Handle user providing their name for order status lookup."""
    clean_input = re.sub(r"[^\w\s]", '', user_input).strip()
    words = clean_input.split()

    skip_words = {'my', 'name', 'is', 'i', 'am', 'call', 'me', 'it', "it's", 'the', 'under'}
    name_words = [w for w in words if w.lower() not in skip_words]

    max_words = CONFIG['max_name_words']
    name = ' '.join(name_words[:max_words]).title() if name_words else clean_input.title()
    context.set_name(name)
    context.awaiting = None

    # Simulate order lookup - in production this would query a database
    status_intent = next((i for i in intents if i['tag'] == 'order_status'), None)
    if status_intent and 'responses_lookup' in status_intent:
        return random.choice(status_intent['responses_lookup']).format(name=name)

    return f"Found it, {name}! Your order is being prepared. Should be ready in about 10-15 minutes!"

def handle_item_input(user_input: str, context: ConversationContext, intent: Dict) -> str:
    """Handle user providing pizza item when awaiting."""
    order_intent = next((i for i in intents if i['tag'] == 'order'), None)
    if not order_intent:
        return CONFIG['order_error_message']

    extracted = extract_order_details(user_input, order_intent)

    if extracted['item']:
        context.awaiting = None
        if not context.customer_name:
            context.pending_item = extracted['item']
            context.pending_size = extracted.get('size', CONFIG['default_size'])
            context.awaiting = 'name'
            return random.choice(order_intent['name_prompt'])

        context.set_order(extracted['item'], extracted.get('size', CONFIG['default_size']))
        context.calculate_price(order_intent)
        values = context.get_context_values()
        return random.choice(order_intent['responses']).format(**values)

    return random.choice(order_intent['confirm_item_prompt'])

def handle_order_intent(intent: Dict, context: ConversationContext, user_input: str) -> str:
    """Handle order placement intent."""
    extracted = extract_order_details(user_input, intent)

    if not extracted['item']:
        context.awaiting = 'item'
        return random.choice(intent['confirm_item_prompt'])

    if not context.customer_name:
        context.pending_item = extracted['item']
        context.pending_size = extracted.get('size', CONFIG['default_size'])
        context.awaiting = 'name'
        return random.choice(intent['name_prompt'])

    size = extracted.get('size', CONFIG['default_size'])
    context.set_order(extracted['item'], size)
    context.calculate_price(intent)

    values = context.get_context_values()
    return random.choice(intent['responses']).format(**values)

def handle_status_intent(intent: Dict, context: ConversationContext) -> str:
    """Handle order status check intent."""
    # If user has active order in this session, show it
    if context.has_active_order:
        values = context.get_context_values()
        return random.choice(intent['responses']).format(**values)

    # Otherwise, ask for name to look up order
    context.awaiting = 'name_for_status'
    return random.choice(intent['responses_no_order'])

def handle_goodbye_intent(intent: Dict, context: ConversationContext) -> str:
    """Handle goodbye/exit intent."""
    if context.customer_name:
        values = context.get_context_values()
        return random.choice(intent['responses']).format(**values)
    return random.choice(intent['responses_no_name'])

print("Response generation functions defined.")

Response generation functions defined.


## 8. Main Chatbot Loop

The interactive chatbot loop:
1. Accepts user input
2. Matches input to intent using regex patterns
3. Generates context-aware response
4. Maintains conversation state
5. Terminates only on `exit` or `quit`

In [8]:
def run_chatbot():
    """Main chatbot interaction loop. Type 'exit' or 'quit' to end."""
    print("="*60, flush=True)
    print("       Welcome to Pizza Palace Chatbot!", flush=True)
    print("="*60, flush=True)
    print("\nI'm your virtual assistant for pizza ordering.", flush=True)
    print("You can ask about our menu, place orders, or check order status.", flush=True)
    print("Type 'exit' or 'quit' to end the conversation.\n", flush=True)
    print("-"*60, flush=True)

    context = ConversationContext()

    while True:
        try:
            user_input = input("\nYou: ").strip()

            if not user_input:
                print(f"Bot: {CONFIG['empty_input_message']}", flush=True)
                continue

            # Echo user input for logging
            print(f"You: {user_input}", flush=True)

            if user_input.lower() in ['exit', 'quit']:
                goodbye_intent = next((i for i in intents if i['tag'] == 'goodbye'), None)
                if goodbye_intent:
                    response = handle_goodbye_intent(goodbye_intent, context)
                    print(f"Bot: {response}", flush=True)
                else:
                    print(f"Bot: {CONFIG['fallback_goodbye']}", flush=True)
                print("\n" + "="*60, flush=True)
                print("           Session Ended. Enjoy your pizza!", flush=True)
                print("="*60, flush=True)
                break

            matched_intent, match = match_intent(user_input, intents)

            if matched_intent:
                response = generate_response(matched_intent, context, user_input)
            else:
                response = CONFIG['no_match_response']

            print(f"Bot: {response}", flush=True)

        except KeyboardInterrupt:
            print(f"\n\nBot: {CONFIG['interrupt_goodbye']}", flush=True)
            break
        except Exception as e:
            print(f"\nBot: {CONFIG['error_message']} ({e})", flush=True)


## 9. Start the Chatbot

- Type your message and press Enter
- Type `exit` or `quit` to end the session

In [9]:
run_chatbot()

       Welcome to Pizza Palace Chatbot!

I'm your virtual assistant for pizza ordering.
You can ask about our menu, place orders, or check order status.
Type 'exit' or 'quit' to end the conversation.

------------------------------------------------------------
You: hey
Bot: Hi there! Welcome to Pizza Palace! Would you like to see our menu or place an order?
You: quit
Bot: Thanks for stopping by! See you next time!

           Session Ended. Enjoy your pizza!


---

## 10. Automated Testing

The following tests demonstrate the chatbot's behavior with various inputs.

In [10]:
def run_test_conversation(test_inputs: List[str], test_name: str = "Test"):
    """Run a test conversation with predefined inputs."""
    print(f"\n{'='*60}")
    print(f"  TEST CASE: {test_name}")
    print(f"{'='*60}")

    test_context = ConversationContext()

    for user_input in test_inputs:
        print(f"\nUser: {user_input}")

        if user_input.lower() in ['exit', 'quit']:
            goodbye_intent = next((i for i in intents if i['tag'] == 'goodbye'), None)
            response = handle_goodbye_intent(goodbye_intent, test_context)
            print(f"Bot: {response}")
            print("[Session ended]")
            break

        matched_intent, match = match_intent(user_input, intents)
        intent_tag = matched_intent['tag'] if matched_intent else 'None'
        response = generate_response(matched_intent, test_context, user_input)

        print(f"[Intent: {intent_tag}]")
        print(f"Bot: {response}")

        if test_context.customer_name or test_context.has_active_order:
            print(f"[Context: name={test_context.customer_name}, order={test_context.current_order['item']}, awaiting={test_context.awaiting}]")

    print(f"\n{'-'*60}")
    return test_context

### Test Case 1: Complete Order Flow

This test demonstrates a complete pizza ordering interaction from greeting to order placement.

In [11]:
test1_inputs = [
    "Hello!",
    "What's on the menu?",
    "I'd like a large pepperoni pizza",
    "John",
    "Where is my order?",
    "exit"
]

run_test_conversation(test1_inputs, "Complete Order Flow")


  TEST CASE: Complete Order Flow

User: Hello!
[Intent: greeting]
Bot: Greetings! I'm your Pizza Palace assistant. Ready to order a delicious pizza?

User: What's on the menu?
[Intent: menu]
Bot: Our delicious pizzas:

- Margherita ($10)
- Pepperoni ($12)
- Hawaiian ($12)
- Veggie Supreme ($13)
- Meat Lovers ($14)
- BBQ Chicken ($13)

Available in Small, Medium (+$2), or Large (+$4).

Which one sounds good?

User: I'd like a large pepperoni pizza
[Intent: order]
Bot: Great choice! What name should I put the order under?

User: John
[Intent: fallback]
Bot: John, your Large Pepperoni pizza is in the queue! Total: $16.00. Ready in about 15-20 minutes.
[Context: name=John, order=Pepperoni, awaiting=None]

User: Where is my order?
[Intent: order_status]
Bot: Your Large Pepperoni pizza is being prepared, John. Should be ready in about 15-20 minutes!
[Context: name=John, order=Pepperoni, awaiting=None]

User: exit
Bot: Thank you for your order, John! Enjoy your pizza!
[Session ended]

------

### Test Case 2: Order with Variations

This test shows different phrasing patterns for ordering and the chatbot's handling of size defaults.

In [12]:
test2_inputs = [
    "Hey there!",
    "Can I get a hawaiian pizza please",
    "Sarah",
    "Actually, I'll have a small margherita too",
    "How long until my order is ready?",
    "quit"
]

run_test_conversation(test2_inputs, "Order with Variations")


  TEST CASE: Order with Variations

User: Hey there!
[Intent: greeting]
Bot: Hey! Thanks for choosing Pizza Palace! What can I get for you today?

User: Can I get a hawaiian pizza please
[Intent: order]
Bot: Great choice! What name should I put the order under?

User: Sarah
[Intent: fallback]
Bot: Got it, Sarah! Your Medium Hawaiian pizza ($14.00) is now being prepared!
[Context: name=Sarah, order=Hawaiian, awaiting=None]

User: Actually, I'll have a small margherita too
[Intent: order]
Bot: Sarah, your Small Margherita pizza is in the queue! Total: $10.00. Ready in about 15-20 minutes.
[Context: name=Sarah, order=Margherita, awaiting=None]

User: How long until my order is ready?
[Intent: order_status]
Bot: Your Small Margherita pizza is being prepared, Sarah. Should be ready in about 15-20 minutes!
[Context: name=Sarah, order=Margherita, awaiting=None]

User: quit
Bot: Thanks Sarah! Enjoy your meal!
[Session ended]

------------------------------------------------------------


### Test Case 3: Menu and Status Without Order

This test shows how the chatbot handles status checks without an active order.

In [13]:
test3_inputs = [
    "Good morning!",
    "Show me the menu",
    "What pizza types do you have?",
    "Where is my order?",
    "Mike",
    "exit"
]

run_test_conversation(test3_inputs, "Menu Browsing and Late Order")


  TEST CASE: Menu Browsing and Late Order

User: Good morning!
[Intent: greeting]
Bot: Welcome to Pizza Palace! I'm here to help you with your pizza order!

User: Show me the menu
[Intent: menu]
Bot: Our delicious pizzas:

- Margherita ($10)
- Pepperoni ($12)
- Hawaiian ($12)
- Veggie Supreme ($13)
- Meat Lovers ($14)
- BBQ Chicken ($13)

Available in Small, Medium (+$2), or Large (+$4).

Which one sounds good?

User: What pizza types do you have?
[Intent: menu]
Bot: Here's our pizza menu:

**Pizzas:**
- Margherita ($10) - Classic tomato and mozzarella
- Pepperoni ($12) - Loaded with pepperoni
- Hawaiian ($12) - Ham and pineapple
- Veggie Supreme ($13) - Bell peppers, mushrooms, olives, onions
- Meat Lovers ($14) - Pepperoni, sausage, bacon, ham
- BBQ Chicken ($13) - Grilled chicken with BBQ sauce

**Sizes:** Small (+$0), Medium (+$2), Large (+$4)

What would you like to order?

User: Where is my order?
[Intent: order_status]
Bot: I can look up your order. What name is it under?

User

### Test Case 4: Fallback Handling

This test demonstrates how the chatbot handles unrecognized inputs.

In [14]:
test4_inputs = [
    "Greetings!",
    "What's the weather like?",
    "I want to order",
    "pepperoni",
    "Alex",
    "order status",
    "exit"
]

run_test_conversation(test4_inputs, "Fallback and Recovery")


  TEST CASE: Fallback and Recovery

User: Greetings!
[Intent: greeting]
Bot: Hey! Thanks for choosing Pizza Palace! What can I get for you today?

User: What's the weather like?
[Intent: menu]
Bot: Our delicious pizzas:

- Margherita ($10)
- Pepperoni ($12)
- Hawaiian ($12)
- Veggie Supreme ($13)
- Meat Lovers ($14)
- BBQ Chicken ($13)

Available in Small, Medium (+$2), or Large (+$4).

Which one sounds good?

User: I want to order
[Intent: order]
Bot: Which pizza would you like? Options: Margherita, Pepperoni, Hawaiian, Veggie Supreme, Meat Lovers, or BBQ Chicken.

User: pepperoni
[Intent: fallback]
Bot: Great choice! What name should I put the order under?

User: Alex
[Intent: fallback]
Bot: Perfect! Medium Pepperoni pizza for Alex, total $14.00. We're firing up the oven!
[Context: name=Alex, order=Pepperoni, awaiting=None]

User: order status
[Intent: order_status]
Bot: Your Medium Pepperoni pizza is being prepared, Alex. Should be ready in about 15-20 minutes!
[Context: name=Alex,

### Test Case 5: Complete Multi-Service Interaction

This comprehensive example demonstrates a realistic customer journey using **all five intents** across multiple conversation turns:

1. **Greeting** - Customer initiates conversation
2. **Menu Inquiry** - Customer browses available options  
3. **Order Placement** - Customer places order with name capture and item extraction
4. **Order Status** - Customer checks on their order
5. **Second Order** - Customer adds another item (context retained)
6. **Status Update** - Customer checks updated order
7. **Goodbye** - Customer ends session

This test showcases:
- Context persistence (name remembered across turns)
- Template substitution with dynamic values
- State transitions between intents
- Price calculation with size modifiers

In [15]:
test5_inputs = [
    "Hi there!",
    "What do you have on the menu?",
    "I'd like to place an order",
    "I'll take a large pepperoni",
    "My name is Maria Garcia",
    "How long until my pizza is ready?",
    "I'd also like a small Hawaiian please",
    "Where's my order now?",
    "Do you deliver to my area?",
    "What sizes are available?",
    "exit"
]

print("="*70)
print("  COMPREHENSIVE MULTI-SERVICE INTERACTION TEST")
print("  Demonstrating: Greeting -> Menu -> Order -> Status -> Order -> Exit")
print("="*70)

test_context = ConversationContext()

for i, user_input in enumerate(test5_inputs, 1):
    print(f"\n[Turn {i}]")
    print(f"Customer: {user_input}")

    if user_input.lower() in ['exit', 'quit']:
        goodbye_intent = next((x for x in intents if x['tag'] == 'goodbye'), None)
        response = handle_goodbye_intent(goodbye_intent, test_context)
        print(f"Bot: {response}")
        print("\n[SESSION ENDED]")
        break

    matched_intent, _ = match_intent(user_input, intents)
    intent_tag = matched_intent['tag'] if matched_intent else 'None'
    response = generate_response(matched_intent, test_context, user_input)

    print(f"[Matched Intent: {intent_tag}]")

    if len(response) > 200:
        print(f"Bot: {response[:200]}...")
        print("     [Response truncated for display]")
    else:
        print(f"Bot: {response}")

    print(f"[Context State]")
    print(f"  - Customer Name: {test_context.customer_name or '(not set)'}")
    print(f"  - Current Order: {test_context.current_order['item'] or '(none)'} ({test_context.current_order['size']})")
    print(f"  - Order Price: ${test_context.current_order['price']:.2f}")
    print(f"  - Has Active Order: {test_context.has_active_order}")
    print(f"  - Awaiting Input: {test_context.awaiting or '(none)'}")

print("\n" + "="*70)
print("  TEST COMPLETE - All 5 intents demonstrated:")
print("  [greeting] [menu] [order] [order_status] [goodbye] + [fallback]")
print("="*70)

  COMPREHENSIVE MULTI-SERVICE INTERACTION TEST
  Demonstrating: Greeting -> Menu -> Order -> Status -> Order -> Exit

[Turn 1]
Customer: Hi there!
[Matched Intent: greeting]
Bot: Hi there! Welcome to Pizza Palace! Would you like to see our menu or place an order?
[Context State]
  - Customer Name: (not set)
  - Current Order: (none) (Medium)
  - Order Price: $0.00
  - Has Active Order: False
  - Awaiting Input: (none)

[Turn 2]
Customer: What do you have on the menu?
[Matched Intent: menu]
Bot: Here's our pizza menu:

**Pizzas:**
- Margherita ($10) - Classic tomato and mozzarella
- Pepperoni ($12) - Loaded with pepperoni
- Hawaiian ($12) - Ham and pineapple
- Veggie Supreme ($13) - Bell pepp...
     [Response truncated for display]
[Context State]
  - Customer Name: (not set)
  - Current Order: (none) (Medium)
  - Order Price: $0.00
  - Has Active Order: False
  - Awaiting Input: (none)

[Turn 3]
Customer: I'd like to place an order
[Matched Intent: order]
Bot: Which pizza would you li

## 11. Pattern Matching Demonstration

This section shows how different input variations are matched to intents.

In [16]:
pattern_tests = [
    ("Hi", "greeting"),
    ("HELLO", "greeting"),
    ("Good morning!", "greeting"),
    ("Hey there", "greeting"),
    ("menu", "menu"),
    ("What do you have?", "menu"),
    ("Show me the menu please", "menu"),
    ("What kinds of pizza do you have?", "menu"),
    ("I'd like a pepperoni pizza", "order"),
    ("Can I get a large hawaiian", "order"),
    ("One meat lovers please", "order"),
    ("I'll have the veggie supreme", "order"),
    ("Where is my order?", "order_status"),
    ("How long until my pizza is ready?", "order_status"),
    ("Order status", "order_status"),
    ("exit", "goodbye"),
    ("quit", "goodbye"),
]

print("Pattern Matching Verification:")
print("-" * 60)
print(f"{'Input':<40} {'Expected':<12} {'Actual':<12} {'Match'}")
print("-" * 60)

all_passed = True
for test_input, expected_intent in pattern_tests:
    matched, _ = match_intent(test_input, intents)
    actual = matched['tag'] if matched else 'None'
    passed = actual == expected_intent
    status = "PASS" if passed else "FAIL"
    if not passed:
        all_passed = False
    print(f"{test_input:<40} {expected_intent:<12} {actual:<12} {status}")

print("-" * 60)
print(f"\nAll tests passed: {all_passed}")

Pattern Matching Verification:
------------------------------------------------------------
Input                                    Expected     Actual       Match
------------------------------------------------------------
Hi                                       greeting     greeting     PASS
HELLO                                    greeting     greeting     PASS
Good morning!                            greeting     greeting     PASS
Hey there                                greeting     greeting     PASS
menu                                     menu         menu         PASS
What do you have?                        menu         menu         PASS
Show me the menu please                  menu         menu         PASS
What kinds of pizza do you have?         menu         menu         PASS
I'd like a pepperoni pizza               order        order        PASS
Can I get a large hawaiian               order        order        PASS
One meat lovers please                   order        

## 12. Summary

### Architecture Overview

The chatbot separates **conversational framework** from **business logic**:

| Layer | Components | Responsibility |
|-------|------------|----------------|
| **Framework** | Intent matching, response generation, dialog flow | Domain-agnostic conversation handling |
| **Business Logic** | `ConversationContext`, price calculation, order validation | Domain-specific operations |
| **Configuration** | `CONFIG` dict in notebook | Runtime settings |
| **Intent Data** | `intents.json` | Patterns, responses, prices (no defaults) |

### Configuration Options

All runtime settings are defined in the `CONFIG` dictionary:

| Setting | Description |
|---------|-------------|
| `default_size` | Default pizza size when not specified |
| `case_insensitive` | Enable case-insensitive pattern matching |
| `max_name_words` | Maximum words to extract from customer name |
| `preprocessing.expand_contractions` | Expand "I'd" to "I would", etc. |
| `preprocessing.lowercase` | Convert input to lowercase |
| `preprocessing.remove_punctuation` | Strip punctuation from input |
| `preprocessing.stem` | Apply Porter stemming |
| `preprocessing.remove_stopwords` | Remove common words |
| `empty_input_message` | Response for empty input |
| `error_message` | Response for exceptions |
| `name_only_response` | Response when name given without order |
| `order_error_message` | Response when order processing fails |
| `fallback_goodbye` | Goodbye when intent not found |
| `interrupt_goodbye` | Goodbye on keyboard interrupt |
| `no_match_response` | Response when no intent matches |

### Business Logic Injection Point

The `ConversationContext` class serves as the integration layer where domain-specific logic is injected:

- `calculate_price()` - Pricing rules, discounts, tax calculation
- `set_order()` - Order validation, inventory checks
- `get_context_values()` - Data formatting for responses